In [1]:
import pandas as pd
import requests
import praw
from praw.models import MoreComments, comment_forest
import pysentiment2 as ps
import demoji
from collections import OrderedDict
from collections import defaultdict
from prawcore.exceptions import Forbidden
from textblob import TextBlob
import nltk
from twitter_scraper import get_tweets
import tweepy

In [2]:
reddit = praw.Reddit(client_id="***",
                     client_secret="***",
                     redirect_uri="***",
                     password="***",
                     user_agent="***",
                     username="***",
                     )

In [3]:
print(reddit.user.me())

sustainablesentiment


In [4]:
targetsubreddits = ["greeninvestor", "sustainability", "climateoffensive", "esginvesting"]

In [6]:
subcomments = OrderedDict.fromkeys(targetsubreddits,[])

In [7]:
dsubcomments = defaultdict(list)

In [8]:
for keys in subcomments:
    for submission in reddit.subreddit(keys).top("all"):
        for top_level_comment in submission.comments[1:]:
            if isinstance(top_level_comment, MoreComments):
                continue
            dsubcomments[keys].append(top_level_comment.body)
            dsubcomments[keys].append(top_level_comment.created_utc)

In [9]:
dsubcomments

defaultdict(list,
            {'greeninvestor': ['Woah!!! This is huge!',
              1580691698.0,
              'Too bad they can’t terminate from all sectors like what the BBC does.',
              1580702805.0,
              'Just yesterday I came across an article on LinkedIn titled [Repeat after me, ‘Canada is uninhabitable without fossil fuels’](https://albertapressleader.ca/repeat-after-me-canada-is-uninhabitable-without-fossil-fuels/).\n\nI went and looked up the profile of both the person who provided the content in the article along with the person who posted it on LinkedIn. Among them they;\n\n- Own a consulting company in the oil and gas sector\n\n- Wrote and published a book about the importance of said sector\n\n- Sit on the board for a few companies in the oil exploration sector\n\n- Had an extensive history of working for companies in the Alberta oil trade\n\nAlways follow the money. It\'s especially sad when you see people gleefully posing with their "I LOVE OIL AND

In [23]:
subcommentsdf = pd.DataFrame().from_dict(dsubcomments, orient = "index")

In [27]:
subcommentsdf = subcommentsdf.transpose()

In [28]:
columns1 = []

for i in targetsubreddits:
    columns1.append(i)
    if columns1 == targetsubreddits:
        for i in targetsubreddits:
            columns1.append(f"{i}-timestamp")

In [29]:
subcommentsdf = pd.DataFrame(subcommentsdf.values.reshape(-1, len(columns1)), columns = [columns1])

In [31]:
order1 = []

for i in targetsubreddits:
    order1.append(i)
    order1.append(i + "-timestamp")

In [32]:
subcommentsdf = subcommentsdf.reindex(columns= [order1])

In [38]:
subcommentsdf

,greeninvestor,greeninvestor-timestamp,sustainability,sustainability-timestamp,climateoffensive,climateoffensive-timestamp,greenfinance,greenfinance-timestamp,esginvesting,esginvesting-timestamp
0,Woah!!! This is huge!,1.58069e+09,There's also the option of just ditching the p...,1.60754e+09,"Whether you're an adult or not, you can [lobby...",1.57613e+09,Yes. Greenwashing is a concern. There were s...,1.45599e+09,Very nicely done,1.6197e+09
1,Too bad they can’t terminate from all sectors ...,1.5807e+09,Wow this is awesome - I remember writing to Co...,1.60754e+09,[deleted],1.57612e+09,Two other coin/block chain projects I was look...,1.57037e+09,Nice work!\nAre you collecting the ratings man...,1.61974e+09
2,Just yesterday I came across an article on Lin...,1.58074e+09,I love the ring tho,1.60754e+09,It’s 3:48 am and all I can think is I gotta wa...,1.57615e+09,You should contact the Climate bond initiative...,1.56859e+09,Done; hope to see the results!,1.61767e+09
3,This made me feel good. \n\nI subscribe to the...,1.58074e+09,How easily can it be dislodged? That looks lik...,1.60754e+09,Worth noting that Person of the Year isn't rea...,1.57617e+09,View in your timezone: \n[today at 17:00 CEST...,1.60259e+09,I have made a video on this topic check it out...,1.61485e+09
4,This is huge. I wonder who will follow.,1.58071e+09,[deleted],1.60754e+09,I did not care about her being TIME's person o...,1.57617e+09,None,NaN,Okay I have been researching this a lot lately...,1.61223e+09
...,...,...,...,...,...,...,...,...,...,...
1251,None,NaN,https://youtu.be/j5v8D-alAKE,1.6179e+09,None,NaN,None,NaN,None,NaN
1252,None,NaN,There was also a company who invested tires wh...,1.6179e+09,None,NaN,None,NaN,None,NaN
1253,None,NaN,Why does the firefighter look like a half life...,1.61791e+09,None,NaN,None,NaN,None,NaN
1254,None,NaN,The only thing we value as a society is your a...,1.61797e+09,None,NaN,None,NaN,None,NaN


In [45]:
body = pd.Series(subcommentsdf[subcommentsdf.columns[::2]].values.ravel())

In [46]:
timestamp = pd.Series(subcommentsdf[subcommentsdf.columns[1::2]].values.ravel())

In [66]:
subcomdf = pd.concat([body.rename("body"), timestamp.rename("timestamp")], axis = 1).dropna()

In [67]:
subcomdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2665 entries, 0 to 6276
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   body       2665 non-null   object
 1   timestamp  2665 non-null   object
dtypes: object(2)
memory usage: 62.5+ KB


In [68]:
targetsearchwords = ["esg", "sustainable investing", "responsible investment", "impact investing", "sustainable finance", "green finance", "corporate social responsibility", "esg investing", "socially responsible investing", "climate funds", "portfolio carbon footprint", "carbon finance", "positive screening", "best in class ESG", "sustainability index", "thematic investing", "triple bottom line"]

In [92]:
searchposts = OrderedDict.fromkeys(targetsearchwords,[])
dsearchwords = defaultdict(list)


In [93]:
for keys in searchposts:
    for sub in reddit.subreddit("all").search(keys):
        dsearchwords[keys].append(sub.title)
        dsearchwords[keys].append(sub.created)

In [94]:
searchwordsdf = pd.DataFrame().from_dict(dsearchwords, orient="index")
searchwordsdf = searchwordsdf.transpose()

In [95]:
columns2 = []

for i in targetsearchwords:
    columns2.append(i)
    if columns2 == targetsearchwords:
        for i in targetsearchwords:
            columns2.append(f"{i}-timestamp")

In [96]:
searchwordsdf = pd.DataFrame(searchwordsdf.values.reshape(-1, len(columns2)), columns = [columns2])

In [97]:
order2 = []

for i in targetsearchwords:
    order2.append(i)
    order2.append(i + "-timestamp")

In [98]:
searchwordsdf = searchwordsdf.reindex(columns= [order2])

In [99]:
body3 = pd.Series(searchwordsdf[searchwordsdf.columns[::2]].values.ravel())

In [100]:
timestamp3 = pd.Series(searchwordsdf[searchwordsdf.columns[1::2]].values.ravel())

In [101]:
searchwddf = pd.concat([body3.rename("body"), timestamp3.rename("timestamp")], axis = 1).dropna()

In [102]:
searchwddf

,body,timestamp
0,As somebody looking to be passively invested a...,1.61876e+09
1,Blackrock hired me to make sustainable investi...,1.61697e+09
2,Shibacoin friends! I found a social network th...,1.61948e+09
3,"First Swell, now COIN - are there reliable imp...",1.61754e+09
4,Bank of America to deploy $1 trillion for sust...,1.618e+09
...,...,...
1694,OANN: China to improve green finance standards...,1.60752e+09
1695,since Hillary likes to screen grab (toxic) pos...,1.61964e+09
1697,Aquaculture companies,1.55792e+09
1698,Investments and Personal Finance Thread - Marc...,1.58453e+09
